In [198]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, SnapshotObjectType, OperationStatusType

In [155]:
# Set the FACE_SUBSCRIPTION_KEY environment variable with your key as the value.
# This key will serve all examples in this document.
KEY = 'cba050788d4e4e69aa869ee84cb6c9e6'

# Set the FACE_ENDPOINT environment variable with the endpoint from your Face service in Azure.
# This endpoint will be used in all examples in this quickstart.
ENDPOINT = 'https://facialrecognition244.cognitiveservices.azure.com/'

In [156]:
# Create an authenticated FaceClient.

face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [157]:
# Detect a face in an image that contains a single face
single_face_image_url = 'https://www.biography.com/.image/t_share/MTQ1MzAyNzYzOTgxNTE0NTEz/john-f-kennedy---mini-biography.jpg'
single_image_name = os.path.basename(single_face_image_url)
detected_faces = face_client.face.detect_with_url(url=single_face_image_url)
if not detected_faces:
    raise Exception('No face detected from image {}'.format(single_image_name))

# Display the detected face ID in the first single-face image.
# Face IDs are used for comparison to faces (their IDs) detected in other images.
print('Detected face ID from', single_image_name, ':')
for face in detected_faces: print (face.face_id)
print()

# Save this ID for use in Find Similar
first_image_face_ID = detected_faces[0].face_id

Detected face ID from john-f-kennedy---mini-biography.jpg :
9b2112dc-1e6a-4b86-98c4-0478bd49b675



In [158]:
# Detect a face in an image that contains a single face
single_face_image_url = 'https://raw.githubusercontent.com/Microsoft/Cognitive-Face-Windows/master/Data/detection1.jpg'
single_image_name = os.path.basename(single_face_image_url)
print(single_image_name)
detected_faces = face_client.face.detect_with_url(url=single_face_image_url)
print(detected_faces)
if not detected_faces:
    raise Exception('No face detected from image {}'.format(single_image_name))

# Convert width height to a point in a rectangle
def getRectangle(faceDictionary):
    rect = faceDictionary.face_rectangle
    left = rect.left
    top = rect.top
    right = left + rect.width
    bottom = top + rect.height
    
    return ((left, top), (right, bottom))


# Download the image from the url
response = requests.get(single_face_image_url)
img = Image.open(BytesIO(response.content))

# For each face returned use the face rectangle and draw a red box.
print('Drawing rectangle around face... see popup for results.')
draw = ImageDraw.Draw(img)
for face in detected_faces:
    draw.rectangle(getRectangle(face), outline='red')

# Display the image in the users default image browser.
img.show()

detection1.jpg
Drawing rectangle around face... see popup for results.


# Find similar faces

In [159]:
# Detect the faces in an image that contains multiple faces
# Each detected face gets assigned a new ID
multi_face_image_url = "http://www.historyplace.com/kennedy/president-family-portrait-closeup.jpg"
multi_image_name = os.path.basename(multi_face_image_url)
detected_faces2 = face_client.face.detect_with_url(url=multi_face_image_url)

In [160]:
# Search through faces detected in group image for the single face from first image.
# First, create a list of the face IDs found in the second image.
second_image_face_IDs = list(map(lambda x: x.face_id, detected_faces2))
# Next, find similar face IDs like the one detected in the first image.
similar_faces = face_client.face.find_similar(face_id=first_image_face_ID, face_ids=second_image_face_IDs)
if not similar_faces[0]:
    print('No similar faces found in', multi_image_name, '.')

In [161]:
# Print the details of the similar faces detected
print('Similar faces found in', multi_image_name + ':')
for face in similar_faces:
    first_image_face_ID = face.face_id
    # The similar face IDs of the single face image and the group image do not need to match, 
    # they are only used for identification purposes in each image.
    # The similar faces are matched using the Cognitive Services algorithm in find_similar().
    face_info = next(x for x in detected_faces2 if x.face_id == first_image_face_ID)
    if face_info:
        print('  Face ID: ', first_image_face_ID)
        print('  Face rectangle:')
        print('    Left: ', str(face_info.face_rectangle.left))
        print('    Top: ', str(face_info.face_rectangle.top))
        print('    Width: ', str(face_info.face_rectangle.width))
        print('    Height: ', str(face_info.face_rectangle.height))

Similar faces found in president-family-portrait-closeup.jpg:
  Face ID:  240cd6ab-85c1-4a37-bc6a-e4e4f7cd1121
  Face rectangle:
    Left:  413
    Top:  57
    Width:  55
    Height:  55


In [162]:
# Used in the Person Group Operations,  Snapshot Operations, and Delete Person Group examples.
# You can call list_person_groups to print a list of preexisting PersonGroups.
# SOURCE_PERSON_GROUP_ID should be all lowercase and alphanumeric. For example, 'mygroupname' (dashes are OK).
PERSON_GROUP_ID = 'my-unique-person-group'

# Used for the Snapshot and Delete Person Group examples.
TARGET_PERSON_GROUP_ID = str(uuid.uuid4()) # assign a random ID (or name it anything)

In [163]:
'''
Create the PersonGroup
'''
# Create empty Person Group. Person Group ID must be lower case, alphanumeric, and/or with '-', '_'.
print('Person group:', PERSON_GROUP_ID)
face_client.person_group.create(person_group_id=PERSON_GROUP_ID, name=PERSON_GROUP_ID)

# Define woman friend
woman = face_client.person_group_person.create(PERSON_GROUP_ID, "Woman")
# Define man friend
man = face_client.person_group_person.create(PERSON_GROUP_ID, "Man")
# Define child friend
child = face_client.person_group_person.create(PERSON_GROUP_ID, "Child")

Person group: my-unique-person-group


APIErrorException: (PersonGroupExists) Person group 'my-unique-person-group' already exists.

In [164]:
'''
Detect faces and register to correct person
'''
# Find all jpeg images of friends in working directory
woman_images = [file for file in glob.glob('*.jpg') if file.startswith("woman")]
man_images = [file for file in glob.glob('*.jpg') if file.startswith("man")]
child_images = [file for file in glob.glob('*.jpg') if file.startswith("child")]

# Add to a woman person
for image in woman_images:
    w = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, woman.person_id, w)

# Add to a man person
for image in man_images:
    m = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, man.person_id, m)

# Add to a child person
for image in child_images:
    ch = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, child.person_id, ch)

In [165]:
'''
Train PersonGroup
'''
print()
print('Training the person group...')
# Train the person group
face_client.person_group.train(PERSON_GROUP_ID)

while (True):
    training_status = face_client.person_group.get_training_status(PERSON_GROUP_ID)
    print("Training status: {}.".format(training_status.status))
    print()
    if (training_status.status is TrainingStatusType.succeeded):
        break
    elif (training_status.status is TrainingStatusType.failed):
        sys.exit('Training the person group has failed.')
    time.sleep(5)


Training the person group...
Training status: failed.



SystemExit: Training the person group has failed.

In [166]:
'''
Identify a face against a defined PersonGroup
'''
# Group image for testing against
group_photo = '/home/furqan/Downloads/datasets/662834_4550818_ertugral_updates.jpg'
IMAGES_FOLDER = os.path.join(os.path.dirname(os.path.realpath(__file__)))
# Get test image
test_image_array = glob.glob(os.path.join(IMAGES_FOLDER, group_photo))
image = open(test_image_array[0], 'r+b')

# Detect faces
face_ids = []
faces = face_client.face.detect_with_stream(image)
for face in faces:
    face_ids.append(face.face_id)

NameError: name '__file__' is not defined

# My Experiment

In [167]:
# def config():
#     # Set the FACE_SUBSCRIPTION_KEY environment variable with your key as the value.
#     # This key will serve all examples in this document.
#     KEY = 'cba050788d4e4e69aa869ee84cb6c9e6'

#     # Set the FACE_ENDPOINT environment variable with the endpoint from your Face service in Azure.
#     # This endpoint will be used in all examples in this quickstart.
#     ENDPOINT = 'https://facialrecognition244.cognitiveservices.azure.com/'
#     return KEY, ENDPOINT

In [168]:
# import requests
# from PIL import Image
# import os
# import numpy as np
# import json
# import matplotlib.pyplot as plt

# image_path = "/home/furqan/Downloads/datasets/662834_4550818_ertugral_updates.jpg"
# image_data = Image.open(image_path)

# image_data.show() 

# subscription_key, face_api_url = config()

# headers = {'Content-Type': 'application/octet-stream',
#            'Ocp-Apim-Subscription-Key': subscription_key}

# params = {
#     'returnFaceId': 'true',
#     'returnFaceLandmarks': 'true',
#     'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion'
# }

# response = requests.post(face_api_url, params=params, headers=headers, data= image_data)
# # print(response.raise_for_status)
# response.raise_for_status()
# faces = response.json()

# print(faces)

In [169]:
# import modules
import requests


KEY = 'cba050788d4e4e69aa869ee84cb6c9e6'
# ENDPOINT = 'https://facialrecognition244.cognitiveservices.azure.com/'

# Request headers set Subscription key which provides access to this API. Found in your Cognitive Services accounts.
headers = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': KEY,
}

body = dict()
body["url"] = "http://www.imagozone.com/var/albums/vedete/Matthew%20Perry/Matthew%20Perry.jpg?m=1355670659"

body = str(body)
print(body, "\n")

# Request URL 
# FaceApiDetect = 'https://westus.api.cognitive.microsoft.com/face/v1.0/detect?returnFaceId=true&returnFaceAttributes=age,gender,headPose,smile,facialHair' 
FaceApiDetect = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/detect?returnFaceId=true&returnFaceAttributes=age,gender,headPose,smile,facialHair'


try:
    # REST Call 
    response = requests.post(FaceApiDetect, data=body, headers=headers) 
    print("RESPONSE:" + str(response.json()))

except Exception as e:
    print(e)

{'url': 'http://www.imagozone.com/var/albums/vedete/Matthew%20Perry/Matthew%20Perry.jpg?m=1355670659'} 

RESPONSE:[{'faceId': 'dece0edd-cd1f-444c-b17e-fb8a0c305025', 'faceRectangle': {'top': 335, 'left': 249, 'width': 319, 'height': 319}, 'faceAttributes': {'smile': 1.0, 'headPose': {'pitch': -6.9, 'roll': -4.8, 'yaw': -0.8}, 'gender': 'male', 'age': 39.0, 'facialHair': {'moustache': 0.1, 'beard': 0.1, 'sideburns': 0.1}}}]


In [170]:
# # import modules
# import requests


# KEY = 'cba050788d4e4e69aa869ee84cb6c9e6'
# # ENDPOINT = 'https://facialrecognition244.cognitiveservices.azure.com/'

# # Request headers set Subscription key which provides access to this API. Found in your Cognitive Services accounts.
# headers = {
# #     'Content-Type': 'application/json',
# #     'Content-Type': 'multipart/form-data',
#     'Content-Type': 'image/jpeg',
#     'Ocp-Apim-Subscription-Key': KEY
    
# }

# # body = dict()

# # path = "/home/furqan/Downloads/datasets/662834_4550818_ertugral_updates.jpg"
# # image_data = Image.open(path)

# # body["url"] = "http://www.imagozone.com/var/albums/vedete/Matthew%20Perry/Matthew%20Perry.jpg?m=1355670659"
# # body["url"] = image_data 


# # files = {
# #     'file': (
# #         os.path.basename('/home/furqan/Downloads/datasets/662834_4550818_ertugral_updates.jpg'), 
# #         open('/home/furqan/Downloads/datasets/662834_4550818_ertugral_updates.jpg', 'rb')
# #     ),
# #     'Content-Type': 'image/jpeg'
# # }

# files = {'media': open('/home/furqan/Downloads/datasets/662834_4550818_ertugral_updates.jpg', 'rb')}


# # body = str(body)
# # print(body, "\n")

# # print(image_data)

# # Request URL 
# # FaceApiDetect = 'https://westus.api.cognitive.microsoft.com/face/v1.0/detect?returnFaceId=true&returnFaceAttributes=age,gender,headPose,smile,facialHair' 
# FaceApiDetect = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/detect?returnFaceId=true&returnFaceAttributes=age,gender,headPose,smile,facialHair'


# try:
#     # REST Call 
#     response = requests.post(FaceApiDetect, files=files, headers=headers) 
#     print("RESPONSE:" + str(response.json()))

# except Exception as e:
#     print(e)

In [184]:
personGroupId="friends"

body = dict()
body["name"] = "F.R.I.E.N.D.S"
body["userData"] = "All friends cast"
body = str(body)

#Request URL 
FaceApiCreateLargePersonGroup = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/persongroups/'+personGroupId 

try:
    # REST Call 
    response = requests.put(FaceApiCreateLargePersonGroup, data=body, headers=headers) 
    print("RESPONSE:" + str(response.status_code))

except Exception as e:
    print(e)

RESPONSE:409


In [185]:
#Request Body
body = dict()
body["name"] = "Chandler"
body["userData"] = "Friends"
body = str(body)

#Request URL 
FaceApiCreatePerson = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/persongroups/'+personGroupId+'/persons' 

try:
    # REST Call 
    response = requests.post(FaceApiCreatePerson, data=body, headers=headers) 
    responseJson = response.json()
    personId = responseJson["personId"]
    print("PERSONID: "+str(personId))
    
except Exception as e:
    print(e)

PERSONID: f4a87a3a-aea9-4338-854f-f9b81f80cd6d


In [177]:
# 5 random images of chandler
chandlerImageList = ["http://www.imagozone.com/var/albums/vedete/Matthew%20Perry/Matthew%20Perry.jpg?m=1355670659",
                     "https://i.pinimg.com/236x/b0/57/ff/b057ff0d16bd5205e4d3142e10f03394--muriel-matthew-perry.jpg",
                     "https://qph.fs.quoracdn.net/main-qimg-74677a162a39c79d6a9aa2b11cc195b1",
                     "https://pbs.twimg.com/profile_images/2991381736/e2160154f215a325b0fc73f866039311_400x400.jpeg",
                     "https://i.pinimg.com/236x/f2/9f/45/f29f45049768ddf5c5d75ff37ffbfb3f--hottest-actors-matthew-perry.jpg"]

#Request URL 
FaceApiCreatePerson = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/persongroups/'+personGroupId+'/persons/'+personId+'/persistedFaces' 

for image in chandlerImageList:
    body = dict()
    body["url"] = image
    body = str(body)

    try:
        # REST Call 
        response = requests.post(FaceApiCreatePerson, data=body, headers=headers) 
        responseJson = response.json()
        persistedFaceId = responseJson["persistedFaceId"]
        print("PERSISTED FACE ID: "+str(persistedFaceId))
    
    except Exception as e:
        print(e)

PERSISTED FACE ID: 9b45e705-92e4-4e39-8fcf-0ae347462830
PERSISTED FACE ID: c8b6f35e-55c2-45c8-bd5b-65c61bf2d18b
PERSISTED FACE ID: 2dbe632d-d781-4a8c-aaa4-42f200836525
PERSISTED FACE ID: 7e14633e-7882-4db8-9701-2b90d2e45f6b
PERSISTED FACE ID: 6d6e4127-ac4d-456f-9033-ad405adbbcfe


In [186]:
#Request Body
body = dict()

#Request URL 
FaceApiTrain = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/persongroups/'+personGroupId+'/train'

try:
    # REST Call 
    response = requests.post(FaceApiTrain, data=body, headers=headers) 
    print("RESPONSE:" + str(response.status_code))

except Exception as e:
    print(e)

RESPONSE:202


In [1]:
# Request Body
body = dict()
body["url"] = "https://upload.wikimedia.org/wikipedia/en/thumb/6/6c/Matthew_Perry_as_Chandler_Bing.jpg/220px-Matthew_Perry_as_Chandler_Bing.jpg"
body = str(body)

# Request URL 
FaceApiDetect = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/detect?returnFaceId=true' 

try:
# REST Call 
    response = requests.post(FaceApiDetect, data=body, headers=headers) 
    responseJson = response.json()
    faceId = responseJson[0]["faceId"]
    print("FACE ID: "+str(faceId))

except Exception as e:
    print(e)

name 'requests' is not defined


In [196]:
faceId

NameError: name 'faceId' is not defined

In [193]:
faceIdsList = [faceId]

# Request Body
body = dict()
body["personGroupId"] = personGroupId
body["faceIds"] = faceIdsList
body["maxNumOfCandidatesReturned"] = 1 
body["confidenceThreshold"] = 0.5
body = str(body)

# Request URL 
FaceApiIdentify = 'https://westus.api.cognitive.microsoft.com/face/v1.0/identify' 

try:
    # REST Call 
    response = requests.post(FaceApiIdentify, data=body, headers=headers) 
    responseJson = response.json()
    personId = responseJson[0]["candidates"][0]["personId"]
    confidence = responseJson[0]["candidates"][0]["confidence"]
    print("PERSON ID: "+str(personId)+ ", CONFIDENCE :"+str(confidence))
        
except Exception as e:
    print("Could not detect")

NameError: name 'faceId' is not defined

In [197]:
# Request URL 
FaceApiGetPerson = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/persongroups/'+personGroupId+'/persons/'+personId

try:
    response = requests.get(FaceApiGetPerson, headers=headers) 
    responseJson = response.json()
    print("This Is "+str(responseJson["name"]))
    
except Exception as e:
    print(e)

This Is Chandler
